In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#Удаляем Name, т.к. бесполезно
train.drop('Name', axis=1, inplace=True)
test.drop('Name', axis=1, inplace=True)

### Смотрим, какие признаки содержат пропуски

In [5]:
total = train.isnull().sum().sort_values(ascending=False)
percent = total / train.shape[0] * 100
statistic = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [6]:
statistic.head()

,Total,Percent
Cabin,687,77.104377
Age,177,19.865320
Embarked,2,0.224467
Fare,0,0.000000
Ticket,0,0.000000


In [7]:
#Удаляем Cabin (слишком много пропусков)
train.drop('Cabin', axis=1, inplace=True)
test.drop('Cabin', axis=1, inplace=True)

In [8]:
#Обьединим кол-во родственников на корабле в один признак

train['family_members_on_board'] = train['SibSp'] + train['Parch']
train.drop(['SibSp', 'Parch', 'Embarked', 'Ticket'], axis=1, inplace=True)

test['family_members_on_board'] = test['SibSp'] + test['Parch']
test.drop(['SibSp', 'Parch', 'Embarked', 'Ticket'], axis=1, inplace=True)

In [9]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [10]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,family_members_on_board
0,1,0,3,male,22.0,7.2500,1
1,2,1,1,female,38.0,71.2833,1
2,3,1,3,female,26.0,7.9250,0
3,4,1,1,female,35.0,53.1000,1
4,5,0,3,male,35.0,8.0500,0


In [11]:
train['Sex'][train['Sex'] == 'male'] = 0
train['Sex'][train['Sex'] == 'female'] = 1

test['Sex'][test['Sex'] == 'male'] = 0
test['Sex'][test['Sex'] == 'female'] = 1

/Users/avgalichin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/avgalichin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/avgalichin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after rem

### Кодируем Embarked при помощи one-hot-encoder'а

In [12]:
X = train.drop('Survived', axis=1)
Y = train['Survived']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40)

In [15]:
#pip install catboost

### Модель

In [16]:
from catboost import CatBoostClassifier

In [17]:
clf = CatBoostClassifier(eval_metric='Accuracy', use_best_model=True, random_seed=42)

In [18]:
clf.fit(X_train, Y_train, eval_set=(X_test, Y_test), early_stopping_rounds=50)

Learning rate set to 0.029139
0:	learn: 0.7851124	test: 0.7932961	best: 0.7932961 (0)	total: 52.3ms	remaining: 52.3s
1:	learn: 0.7823034	test: 0.8100559	best: 0.8100559 (1)	total: 53.7ms	remaining: 26.8s
2:	learn: 0.8089888	test: 0.8324022	best: 0.8324022 (2)	total: 55.6ms	remaining: 18.5s
3:	learn: 0.8160112	test: 0.8268156	best: 0.8324022 (2)	total: 57.1ms	remaining: 14.2s
4:	learn: 0.8160112	test: 0.8268156	best: 0.8324022 (2)	total: 58.2ms	remaining: 11.6s
5:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 59.9ms	remaining: 9.91s
6:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 61ms	remaining: 8.65s
7:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 62ms	remaining: 7.69s
8:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 63.5ms	remaining: 6.99s
9:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 64.5ms	remaining: 6.38s
10:	learn: 0.8075843	test: 0.8268156	best: 0.8324022 (2)	total: 65.5ms	remaining: 5.89s
11:	learn: 0.807

### Submission

In [19]:
test_id = test.PassengerId
pred = clf.predict(test)

In [20]:
submission = pd.DataFrame()
submission['PassengerId'] = test_id
submission['Survived'] = pred.astype(np.int)

submission.to_csv('TitanicSubmission.csv', index=False)